<a href="https://colab.research.google.com/github/y-hiroki-radiotech/llm-final-task/blob/main/llm_jp_3_13b_task_solve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount("/content/drive")
%cd "/content/drive/MyDrive/LLM/2024_大規模言語モデル/05.最終課題/3. task_solve_code"

Mounted at /content/drive
/content/drive/MyDrive/LLM/2024_大規模言語モデル/05.最終課題/3. task_solve_code


UnslothとQLoRAを使ったInstructionファインチューニング

In [2]:
!pip install wandb unsloth trl datasets accelerate bitsandbytes peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### HuggingFace Authentication

In [3]:
from google.colab import userdata

HUGGINGFACE_TOKEN = userdata.get('HF_TOKEN_READ')
!huggingface-cli login --token $HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `LLM_Course` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLM_Course`


### Loading the Base Model

In [4]:
import torch
from unsloth import FastLanguageModel

model_name = "llm-jp/llm-jp-3-13b"

max_seq_length = 1024
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Exception ignored in: <function _xla_gc_callback at 0x790d3bb767a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

KeyboardInterrupt: 

## データセットの準備

In [7]:
from datasets import load_dataset

train_data = load_dataset("hiroki-rad/elyza_generated_data-1511", split="train")
train_data

Dataset({
    features: ['question', 'answer'],
    num_rows: 1511
})

In [14]:
train_data

Dataset({
    features: ['question', 'answer'],
    num_rows: 1511
})

### Loading and Processing the Dataset

## チャットテンプレートの作成

In [58]:
system_prompt = "以下の質問に対して、質問内容をよく分析して、適切な回答をしてください。"

from unsloth.chat_templates import get_chat_template

# Initialize the tokenizer with the chat template and mapping
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"}, # ShareGPT style
    map_eos_token=True, # <|im_end|> to <|eot_id|> instead
)
def formatting_prompts_func(examples):
    convos = []

    # Iterate through each item in the batch
    for question, answer in zip(examples["question"], examples["answer"]):
        tool_user = {
            "content": system_prompt,
            "role": "system"
        }
        query_user = {
            "content": f"質問:{question}",
            "role": "user"
        }
        assistant = {
            "content": f"回答:{answer}",
            "role": "assistant"
        }
        convos.append([tool_user, query_user, assistant])

    texts = [tokenizer.apply_chat_template(
        convo,
        tokenize=False,
        add_generation_prompt=False,
        return_tensors=None,
    ) + tokenizer.eos_token for convo in convos] # add tokenizer.eos_token

    return {"text": texts}


In [59]:
train_data = train_data.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1511 [00:00<?, ? examples/s]

### LoRA可能な線形層を取得する

In [60]:
import re

model_modules = str(model.modules)
pattern = r"\((\w+)\): Linear"
linear_layer_names = re.findall(pattern, model_modules)
target_modules = list(set(linear_layer_names))
print(target_modules)

['default', 'original_module', 'base_layer']


### QLoRAを設定する

In [61]:
r = 128
lora_alpha = 128

model = FastLanguageModel.get_peft_model(
    model,
    target_modules=target_modules,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

TypeError: Unsloth: Your model already has LoRA adapters. Your new parameters are different.

### Defining Training Arguments

In [ ]:
# collatorを使って「応答フォーマット」を指定して、それに続くトークンのみを損失関数の計算対象とする。
# from trl import DataCollatorForCompletionOnlyLM
# response_template_ids = tokenizer.encode("assistant")
# collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)

In [71]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,  # 追加：評価用のバッチサイズ
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    warmup_ratio=0.05,
    learning_rate=2e-4,
    num_train_epochs=2,
    fp16=not is_bfloat16_supported,
    bf16=is_bfloat16_supported,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    max_grad_norm=0.3,
    seed=3407,
    output_dir="./outputs",
    report_to="wandb",
    logging_steps=5,
    logging_strategy="steps",
    # 評価関連のパラメータを追加
    # evaluation_strategy="steps",     # ステップごとに評価を実行
    # eval_steps=5,                 # 100ステップごとに評価を実行
    save_strategy="epoch",          # ステップごとにモデルを保存
    greater_is_better=False,        # メトリクスは小さい方が良いか（lossの場合はFalse）
    save_only_model=False
)

### wandbの設定

In [72]:
config = {
    "output_dir": training_args.output_dir,
    "num_train_epochs": training_args.num_train_epochs,
    "per_device_train_batch_size": training_args.per_device_train_batch_size,
    "bf16": training_args.bf16,
    "optim": training_args.optim,
    "learning_rate": training_args.learning_rate,
    "warmup_ratio": training_args.warmup_ratio,
    "lr_scheduler_type": training_args.lr_scheduler_type
}

In [64]:
# modify
from transformers import TrainerCallback

class LoggingCallback(TrainerCallback):
    def __init__(self, log_every=16):
        self.log_every = log_every

    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.global_step % self.log_every == 0:
            if "loss" in logs:
                wandb.log({"training_loss": logs["loss"]}, step=state.global_step)
            if "eval_loss" in logs:
                wandb.log({"validation_loss": logs["eval_loss"]}, step=state.global_step)

logging_callback = LoggingCallback(log_every=1)

In [65]:
import os
import wandb
from google.colab import userdata

def setup_wandb(project_name: str, run_name: str, config: str, job_type=None):
    # set up your API key
    try:
        WANDB_KEY = userdata.get('WANDB_API_KEY')
        wandb.login(key=WANDB_KEY)
        os.environ["WANDB_ENTITY"] = "y-hiroki-rad"
    except KeyError:
        raise EnvironmentError("WANDB_API_KEY is not set in the environment variables.")
    except Exception as e:
        print(f"Error logging into WandB: {e}")

    # Optional: Log models
    os.environ["WANDB_LOG_MODEL"] = "checkpoint"
    os.environ["WANDB_WATCH"] = "all"
    os.environ["WANDB_SILENT"] = "true"

    # Initialize the WandB run
    try:
        wandb.init(project=project_name, name=run_name, config=config, job_type=job_type)
        print(f"WandB run initialized: Project - {project_name}, Run - {run_name}")
    except Exception as e:
        print(f"Error initializing WandB run: {e}")

In [73]:
proj_name = model_name.replace("/", "-")

project_name = f"{proj_name}-{r}-{lora_alpha}-task_solve-llm-jp-13b"
run_name = "elyza-100-generated_1000-fine-tuning-epoch-2"
job_type = "fine-tuning"

setup_wandb(project_name=project_name, run_name=run_name, config=config, job_type=job_type)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


WandB run initialized: Project - llm-jp-llm-jp-3-13b-128-128-task_solve-llm-jp-13b, Run - elyza-100-generated_1000-fine-tuning-epoch-2


### Training with SFTTrainer and Unsloth

In [74]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer,
    train_dataset = train_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc=2,
    # eval_dataset  = eval_data,
    packing = False,
    args = training_args,
    callbacks = [logging_callback],
    tokenizer = tokenizer
)

Map (num_proc=2):   0%|          | 0/1511 [00:00<?, ? examples/s]

In [68]:
# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
10.029 GB of memory reserved.


In [75]:
from unsloth import unsloth_train

trainer_stats = unsloth_train(trainer)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,511 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 4
\        /    Total batch size = 64 | Total steps = 46
 "-____-"     Number of trainable parameters = 1,010,565,120


Step,Training Loss
5,0.950600
10,1.003100
15,0.990400
20,0.916800
25,0.911700
30,0.651400
35,0.643700
40,0.630700
45,0.638100


wandb: Adding directory to artifact (./outputs/checkpoint-23)... Done. 20.7s
wandb: Adding directory to artifact (./outputs/checkpoint-46)... Done. 17.4s


KeyboardInterrupt: 

In [76]:
wandb.finish()

train/epoch,▁▂▃▄▅▅▆▇█
train/global_step,▁▂▃▄▅▅▆▇██
train/grad_norm,█▅▄▃▁▁▃▂▂
train/learning_rate,█▇▆▅▅▄▃▂▁
train/loss,▇██▆▆▁▁▁▁
training_loss,▇██▆▆▁▁▁▁
train/epoch,1.89474
train/global_step,45
train/grad_norm,0.78344
train/learning_rate,0.0
train/loss,0.6381


### モデルをマージしてHuggingFaceにPushする

In [4]:
import torch
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

In [5]:
%ls

elyza_tasks_100_labels.csv     llm-jp-3-13b-task_solve.ipynb  _unsloth_temporary_saved_buffers/
huggingface_tokenizers_cache/  outputs/                       wandb/
label_data_all.csv             prompt-engineering-ft/         合成データ3000-訓練/


In [6]:
r = 128
lora_alpha = 128

model_name = "llm-jp/llm-jp-3-13b"
adapter = "outputs/checkpoint-46"
output_dir = f"./{model_name}-{r}-epoch-2-ft"

In [7]:
model = AutoModelForCausalLM.from_pretrained(
          model_name,  device_map={"": 0}, torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

In [8]:
model = PeftModel.from_pretrained(model, adapter)
model = model.merge_and_unload()

In [9]:
HF_TOKEN_WRITE = userdata.get("HF_TOKEN_WRITE")

!huggingface-cli login --token $HF_TOKEN_WRITE

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `LLM_new_token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLM_new_token`


In [10]:
repo_id = f'hiroki-rad/{model_name.replace("/", "-")}-{r}-ft-1500'

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

model-00001-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/2.71G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hiroki-rad/llm-jp-llm-jp-3-13b-128-ft-1500/commit/df62c966fec776323cd61e5dd3aa97c63e67e0ec', commit_message='Upload tokenizer', commit_description='', oid='df62c966fec776323cd61e5dd3aa97c63e67e0ec', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hiroki-rad/llm-jp-llm-jp-3-13b-128-ft-1500', endpoint='https://huggingface.co', repo_type='model', repo_id='hiroki-rad/llm-jp-llm-jp-3-13b-128-ft-1500'), pr_revision=None, pr_num=None)